# Crawling Instagram Posts

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
### Import Libraries ###
import pandas as pd
import numpy as np
import time
import requests
import urllib.request
from urllib.parse import quote_plus
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import unicodedata

In [3]:
### Import Hashtag List ###
tag_table = pd.read_excel('HashtagList.xlsx', engine='openpyxl')
tag_table

,일상,육아,여행/장소,취미/예술,패션,음식,애완동물
0,일상,육아,대구,럽스타그램,데일리룩,먹스타그램,멍스타그램
1,데일리,육아스타그램,홍대,사진,오오티디,맛스타그램,고양이
2,셀카,육아소통,강남,사랑해,패션,카페,반려견
3,셀스타그램,딸스타그램,광주,운동,옷스타그램,먹방,강아지
4,셀피,육아맘,해운대,네일아트,NaN,맛집,냥스타그램
5,얼스타그램,맘스타그램,제주도,젤네일,NaN,커피,개스타그램
6,일상스타그램,아들스타그램,서면,인테리어,NaN,카페스타그램,펫스타그램
7,주말,도치맘,대전,운동하는여자,NaN,NaN,댕댕이
8,다이어트,애스타그램,이태원,네일,NaN,NaN,NaN
9,데이트,인스타베이비,제주도,방탄소년단,NaN,NaN,NaN


In [4]:
### Settings ###

# 이미지를 저장할 'img'라는 디렉토리를 미리 생성해주세요.

path = '/Users/kimchaehyeong/Downloads/chromedriver' # 크롬드라이버 위치

ID = '01088797388' # 인스타 id
PW = 'jc041794' # 인스타 비밀번호

SCROLL_NUM = 13 # 얼마나 스크롤 할 지 (스크롤을 내려 게시물을 많이 확보해야 합니다. 값이 최소 1이어야 합니다.)
CRAWLING_NUM = 100 # 크롤링 할 게시물 개수

tag_list = tag_table['애완동물'] #검색할 태그 column명과 인덱싱 해주세요.
tag_list.dropna(inplace=True)
tag_list

0    멍스타그램
1      고양이
2      반려견
3      강아지
4    냥스타그램
5    개스타그램
6    펫스타그램
7      댕댕이
Name: 애완동물, dtype: object

In [5]:
def get_url(driver, tag):
    url = "https://www.instagram.com/explore/tags/"
    url = url + quote_plus(tag)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = bs(html, 'lxml')

In [6]:
def scroll(drive, scroll_num):
    
    # SCROLL_NUM만큼 맨 밑으로 스크롤하고 맨 위로 돌아오기
    for _ in range(scroll_num):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
        
    driver.execute_script('window.scrollTo(0,0);')

    # 첫번째 게시물
    driver.find_element_by_class_name('_9AhH0').click()

In [7]:
def crawl(driver, crawl_num, tag):
    
    n = 0
    
    for i in range(crawl_num):
        
        html = driver.page_source
        soup = bs(html, 'lxml')
        time.sleep(1)

        # 포스팅 이미지 및 이미지 url 저장
        img_url = driver.find_elements_by_class_name('FFVAD')[-1].get_property('src')
        image_name = tag + '_' + str(n) + '.jpg'
        with urllib.request.urlopen(img_url) as f:
            with open('./img/' + tag + '_' + str(n) + '.jpg', 'wb') as h:
                img = f.read()
                h.write(img)

        # 포스팅 장소 저장
        location = driver.find_elements_by_class_name('O4GlU')
        if len(location) == 0:
            location = np.nan
        else:
            location = location[0].text
            location = unicodedata.normalize('NFC', location)

        # 포스팅 내용 저장
        contents = driver.find_elements_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span')
        if len(contents) == 0:
            contents = np.nan
        else:
            contents = contents[0].text
            contents = unicodedata.normalize('NFC', contents)

        # 답글 열기
        replies = driver.find_elements_by_class_name('EizgU')
        for reply in replies:
            reply.click()
        time.sleep(2)
        
        # 해시태그 저장
        hashtags = ''
        tag_list = driver.find_elements_by_class_name('xil3i')
        if len(tag_list) == 0:
            hashtags = np.nan
        else:    
            for k in range(len(tag_list)):
                hashtags += tag_list[k].text
            hashtags = unicodedata.normalize('NFC', hashtags)
        
        # 수집한 모든 정보 추가
        image_url_list.append(img_url)
        image_name_list.append(image_name)
        location_list.append(location)
        hashtags_list.append(hashtags)
        contents_list.append(contents)

        # 다음 게시물
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(1)
        
        n += 1

In [8]:
### START CRAWLING !! ###
image_name_list = []
image_url_list = []
contents_list = []
location_list = []
hashtags_list = []

driver = webdriver.Chrome(path)
driver.maximize_window()
driver.get('https://instagram.com/explore')
time.sleep(1)

driver.find_element_by_name('username').send_keys(ID)
driver.find_element_by_name('password').send_keys(PW)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button').click()
time.sleep(3)

for tag in tag_list:
    get_url(driver, tag)
    time.sleep(1)
    
    scroll(driver, SCROLL_NUM)
    crawl(driver, CRAWLING_NUM, tag)

In [9]:
### Save Dataset ###
data = {'image_name':image_name_list,
        'image_url':image_url_list,
         'content':contents_list,
         'location':location_list,
         'hashtags':hashtags_list}
df = pd.DataFrame(data)
df = df.drop_duplicates(['image_url']) # 이미지 url을 기준으로 중복 제거
df.to_csv('data.csv', index=True, encoding='utf-8-sig')

In [10]:
df

,image_name,image_url,content,location,hashtags
0,멍스타그램_0.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,_👻\n\n냄새가 난다\n.\n.\n.\n.\n#한남동#비숑요셉#개춘기#개린이#댕댕...,NaN,#한남동#비숑요셉#개춘기#개린이#댕댕이#반려견#얼짱#강아지#견생#일상#강아지모델#비...
1,멍스타그램_1.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,"모델 데뷔해 번 300만원, 유기견 쉼터에 플렉스한 '명예 택배기사' 경태\n\n#...",NaN,#경태#강아지#명예택배기사#기부#반려견#반려동물#댕댕이#멍스타그램#펫스타그램#CJ대...
2,멍스타그램_2.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,"내가 먹을건 1도 없는 을왕리😵 터키츄도 안챙긴 어매는 사진만 자꾸 찍고,, 아이구...",을왕리 카페오라,#입모양ㅅ#댕댕이#멍스타그램#강아지그램#비숑#미니비숑#아기비숑#강아지일상#강아지옷#...
3,멍스타그램_3.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,"""반 묶음이 예뻐? 양갈래가 예뻐?""…랜선 집사들 숨멎게 한 강아지\n\n#강아지 ...",NaN,#강아지#댕댕이#반려견#반려동물#헤어스타일#멍스타그램#펫스타그램#pet#cat#dog
4,멍스타그램_4.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,귀여운 푸들 보면서 맛점!😁\n\n푸들분양 문의 DM주세요❤\n\n#푸들분양 #푸들...,NaN,#푸들분양#푸들#푸들스타그램#토이푸들#서울강아지분양#부산강아지분양#대전강아지분양#인...
...,...,...,...,...,...
793,댕댕이_93.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,엄망이 아빵이 시쟝 됴시미 빤니 다뇨와 🤎\n따워리 코하고 이쓰께 ?,NaN,#크림푸들#댕스타그램#푸들#멍스타그램#강아지산책#강아지모임#인스타독#애견인맞팔#반려...
794,댕댕이_94.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,순간포착 📸ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n기여미 퐁 내사룽 퐁❤️\n.\n.\n.\n#...,NaN,#포메퐁이#포메라니안#반려견#댕댕이#강아지#맞팔#멍스타그램#댕댕이그램#펫스타그램#p...
796,댕댕이_96.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,우리 초코 졸려요~~~~😆🥰\n\n#비숑프리제#말티푸#강아지입양#강아지#반려견#비숑...,NaN,#비숑프리제#말티푸#강아지입양#강아지#반려견#비숑입양#강아지그램#반려견그램#일상#소...
798,댕댕이_98.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,그만하고 노라줄ㄹH ? ?,NaN,#반려견#강아지#비숑#비숑스타그램#견스타그램#멍스타그램#daily#멍팔#좋반#좋아요...
